In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("TER").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/18 22:31:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("delimiter", "\t").csv(
    '/kaggle/input/open-food-facts/en.openfoodfacts.org.products.csv',
    header=True
)

In [4]:
from pyspark.sql.functions import col, count

# Calculer le nombre total de lignes
total_rows = df.count()

In [5]:
from pyspark.sql.functions import sum as _sum, when

threshold = 0.75

# Calculer le ratio de nulls pour chaque colonne
null_ratios = df.select([
    (_sum(when(
        col(c).isNull() | (col(c) == "") | (col(c) == "null"), 
        1
    ).otherwise(0)) / total_rows).alias(c) 
    for c in df.columns
]).first()

# Colonnes à garder (< 75% de nulls)
columns_to_keep = [c for c in df.columns if null_ratios[c] < threshold]

df_cleaned = df.select(columns_to_keep)

print(f"Conservées: {len(columns_to_keep)}/{len(df.columns)}")

26/02/18 22:31:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Conservées: 69/215


In [6]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, desc, to_timestamp

window = Window.partitionBy("code").orderBy(
    desc(to_timestamp("last_modified_datetime"))
)

df_cleaned = df_cleaned.withColumn("row_num", row_number().over(window)) \
                       .filter(col("row_num") == 1) \
                       .drop("row_num")

In [7]:
window = Window.partitionBy("product_name", "brands").orderBy(desc(to_timestamp("last_modified_datetime")))

df_cleaned = (df_cleaned
    .withColumn("row_num", row_number().over(window))
    .filter(
        (col("row_num") == 1) |
        (col("product_name").isNull() & col("brands").isNull())
    )
    .drop("row_num")
)

In [8]:
df_cleaned = df_cleaned.drop("url", "creator", "created_t", "created_datetime", "last_modified_t", "last_modified_by", "last_updated_t", "last_updated_datetime", "brands", "brands_tags", "categories", "categories_tags", 'labels', 'labels_tags', 'countries', 'countries_tags', 'labels', 'labels_tags', 'countries', 'countries_tags:', 'food_groups', 'food_groups_tags', 'states', 'states_tags', 'last_image_t', 'last_image_datetime', 'main_category', 'image_url', 'image_small_url', 'image_ingredients_url', 'image_ingredients_small_url', 'image_nutrition_url', 'image_nutrition_small_url')

In [9]:
import os
import shutil

output_dir = "/kaggle/working/output_temp"

# Utiliser TAB comme séparateur
df_cleaned.coalesce(1).write \
    .option("header", "true") \
    .option("sep", "\t") \
    .option("quote", "") \
    .option("escape", "\\") \
    .mode("overwrite") \
    .csv(output_dir)

# Trouver le fichier part-*.csv et le renommer
csv_file = [f for f in os.listdir(output_dir) if f.startswith("part-")][0]
shutil.move(f"{output_dir}/{csv_file}", "/kaggle/working/df.csv")
shutil.rmtree(output_dir)

In [10]:
import pandas as pd
df = pd.read_csv("/kaggle/working/df.csv", sep='\t', on_bad_lines='skip')

/tmp/ipykernel_55/1458899342.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/working/df.csv", sep='\t', on_bad_lines='skip')


In [11]:
df.head(20)

,code,last_modified_datetime,product_name,quantity,brands_en,categories_en,labels_en,countries_en,ingredients_text,ingredients_tags,...,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,nutrition-score-fr_100g
0,3664798010404,2022-02-12T05:03:20Z,NaN,NaN,1-60,NaN,NaN,Cambodia,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20240035,2025-09-26T14:52:22Z,NaN,Schoko Waffelröllchn Vollmilch,2024-00-35,"Snacks,Sweet snacks,Biscuits and cakes,Biscuit...","Fair trade,German Agricultural Society,Fairtra...",Germany,Delicia 20240035,de:delicia-20240035,...,32.000,18.000,39.00,19.00,9.20,9.70,0.040000,0.01600,0.0,16.0
2,3830081921584,2025-08-09T16:24:28Z,NaN,75g,4endurance,"Dietary supplements,fr:gel-energetique","No gluten,Vegetarian,Vegan",France,NaN,NaN,...,0.000,NaN,60.00,29.30,NaN,0.00,0.955000,0.38200,NaN,NaN
3,2938660012454,2023-03-07T08:36:59Z,NaN,"1,245 kg","A-queijeira,Lactimaf","Dairies,Fermented foods,Fermented milk product...",Green Dot,Portugal,_Leite_ de vaca pasteurizado (Origem: Portugal...,"en:pasteurised-cow-s-milk,en:dairy,en:milk,en:...",...,32.000,23.000,1.40,0.20,NaN,22.20,1.500000,0.60000,0.0,14.0
4,8858638001102,2024-12-14T03:07:24Z,NaN,322 ml,"Aje-big,อาเจบิ๊ก",th:เครื่องดืมอัดก๊าซ,NaN,Thailand,"น่าตาล 4.4%, กลิ่นส้ม 0.13%, ไฟเบอร์จากข้าวโพด...","th:น่าตาล,th:กลิ่นส้ม,th:ไฟเบอร์จากข้าวโพด,th:...",...,0.000,0.000,4.35,4.35,NaN,NaN,0.000000,0.00000,0.0,NaN
5,8004521900056,2025-09-04T16:00:37Z,NaN,NaN,antico-pastificio-tealdi-srl,NaN,NaN,Germany,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8413760002955,2025-05-26T21:47:41Z,NaN,225 g,arruabarrena,Frozen foods,Green Dot,France,"Farine de BLÉ, margarine végétale graisses vég...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...",...,31.000,15.000,59.00,20.00,1.80,5.40,0.920000,0.36800,0.0,25.0
7,9002707508385,2025-09-04T16:00:57Z,NaN,NaN,ablinger,NaN,NaN,Germany,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4250752204560,2024-07-30T18:37:27Z,NaN,56l,abtei,NaN,NaN,Germany,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,8414606401093,2023-09-19T19:55:57Z,NaN,NaN,acebuchal-del-sur,NaN,NaN,Spain,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3508806 entries, 0 to 3508805
Data columns (total 40 columns):
 #   Column                                                 Dtype  
---  ------                                                 -----  
 0   code                                                   object 
 1   last_modified_datetime                                 object 
 2   product_name                                           object 
 3   quantity                                               object 
 4   brands_en                                              object 
 5   categories_en                                          object 
 6   labels_en                                              object 
 7   countries_en                                           object 
 8   ingredients_text                                       object 
 9   ingredients_tags                                       object 
 10  ingredients_analysis_tags                              object 
 11

In [19]:
df.describe()

,serving_quantity,additives_n,nutriscore_score,nova_group,product_quantity,unique_scans_n,completeness,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,nutrition-score-fr_100g
count,1.142547e+06,1.077188e+06,1.100048e+06,945062.000000,1.188339e+06,1.141155e+06,3.508573e+06,2.505657e+06,2.548833e+06,2.522316e+06,2.334900e+06,2.526292e+06,2.383038e+06,1.292511e+06,2.526653e+06,2.192767e+06,2.192765e+06,1.074833e+06,1.100048e+06
mean,3.579567e+06,1.860844e+00,1.103483e+01,3.362468,2.635127e+14,3.211775e+00,4.293685e-01,-1.547708e+12,2.421544e+13,1.842183e+19,4.066534e+15,1.965192e+24,4.196324e+39,7.736878e+34,2.302651e+04,6.450342e+00,2.580160e+00,1.738382e+01,1.103483e+01
std,3.524151e+09,2.773121e+00,1.028475e+01,1.015004,2.863419e+17,1.341931e+01,2.128494e-01,6.062457e+15,3.862286e+16,2.925718e+22,6.213818e+18,3.123538e+27,6.477904e+42,8.795953e+37,3.492820e+07,5.700276e+03,2.280111e+03,3.534321e+01,1.028475e+01
min,0.000000e+00,0.000000e+00,-1.800000e+01,1.000000,0.000000e+00,1.000000e+00,5.000000e-02,-8.446744e+18,0.000000e+00,-5.000000e-01,-1.000000e-01,-1.000000e+00,-1.450000e+00,-4.470000e+01,-1.260000e+01,0.000000e+00,0.000000e+00,-6.588726e+01,-1.800000e+01
25%,3.000000e+01,0.000000e+00,2.000000e+00,3.000000,1.000000e+02,1.000000e+00,2.750000e-01,1.060000e+02,4.440000e+02,1.000000e+00,1.500000e-01,4.100000e+00,8.000000e-01,0.000000e+00,1.800000e+00,8.750000e-02,3.500000e-02,0.000000e+00,2.000000e+00
50%,8.000000e+01,1.000000e+00,1.100000e+01,4.000000,2.500000e+02,1.000000e+00,3.750000e-01,2.540000e+02,1.059000e+03,7.000000e+00,1.900000e+00,1.680000e+01,3.700000e+00,1.700000e+00,6.400000e+00,5.600000e-01,2.240000e-01,2.253606e-02,1.100000e+01
75%,1.210000e+02,3.000000e+00,1.900000e+01,4.000000,4.720000e+02,2.000000e+00,5.750000e-01,3.950000e+02,1.650000e+03,2.000000e+01,7.000000e+00,5.220000e+01,1.700000e+01,4.000000e+00,1.300000e+01,1.352941e+00,5.416667e-01,1.950000e+01,1.900000e+01
max,3.760276e+12,3.190000e+02,5.500000e+01,4.000000,3.121424e+20,3.191000e+03,1.100000e+00,4.554519e+18,6.166162e+19,4.646567e+25,9.494949e+21,4.964649e+30,1.000000e+46,1.000000e+41,5.546251e+10,8.428102e+06,3.371241e+06,1.346100e+04,5.500000e+01
